<a href="https://colab.research.google.com/github/pablogadhi/TranslationDetector/blob/master/sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/pablogadhi/TranslationDetector
!python -m spacy download en
!python -m spacy download es
%cd TranslationDetector
!pip install --upgrade torch torchtext

Cloning into 'TranslationDetector'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 96 (delta 52), reused 63 (delta 28), pack-reused 0
Unpacking objects: 100% (96/96), done.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 16.2MB 541kB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.5-cp36-none-any.whl size=16172936 sha256=f368c612122485be7e93b5df491f6c67d70aac3b685bb0b33ce2faed6b504f41
  Stored in directory: /tmp/pip-ephem-wheel-cache-h8ml_4ub/wheels/05/4f/66/9d0c806f86de08e8645d67996798c49e1512f9c3a250d74242
Successfully built es-core-news-sm
✔ Download and installation successful
You can now load 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!git pull

Already up to date.


In [0]:
import math
import torch
from translation.train import train_model, run_epoch
from translation.data_loader import load_data, make_iters, make_batch
from translation.transformer import Transformer
from translation.utils import LabelSmoothing, LossCompute, DynamicOptimizer
from translation.translate import translate_dataset

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [7]:
!nvidia-smi

Fri Jun 12 00:47:30 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    10W /  70W |     10MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Load data and create the model

In [0]:
DATA_DIR='/content/drive/My Drive/NLP/data/'

In [0]:
SRC, TGT, train, valid, test = load_data(
        DATA_DIR + "en-es-0_", "en.txt", "es.txt", DATA_DIR + "SRC_Field.pt", DATA_DIR + "TGT_Field.pt")

Loading data...
Data loaded!


In [0]:
train_itr, valid_itr, test_itr = make_iters(
        train, valid, test, device, batch_size=4000)
model = Transformer(len(SRC.vocab), len(TGT.vocab)).to(device)

In [0]:
len(SRC.vocab.itos)

40000

### Train the model

In [0]:
train_model(model, train_itr, valid_itr, SRC, TGT, device, 10, save_at=1, checkpoint_f=DATA_DIR + "checkpoint.pt")

### Test saved models

In [23]:
SRC, TGT, train_en, val_en, test_en = load_data(
        DATA_DIR + "en-es-0_", "en.txt", "es.txt", DATA_DIR + "SRC_Field.pt", DATA_DIR + "TGT_Field.pt")

en_model_data = torch.load(DATA_DIR +"en-es_checkpoint_2.pt")
en_model = Transformer(len(SRC.vocab), len(TGT.vocab)).to(device)
en_model.load_state_dict(en_model_data)

train_itr_en, valid_itr_en, test_itr_en = make_iters(
        train_en, val_en, test_en, device, batch_size=4000)

_, _, train_es, val_es, test_es = load_data(
        DATA_DIR + "en-es-0_", "es.txt", "en.txt", DATA_DIR + "TGT_Field.pt", DATA_DIR + "SRC_Field.pt")

es_model_data = torch.load(DATA_DIR +"es-en_checkpoint_2.pt")
es_model = Transformer(len(TGT.vocab), len(SRC.vocab)).to(device)
es_model.load_state_dict(es_model_data)

train_itr_es, valid_itr_es, test_itr_es = make_iters(
        train_es, val_es, test_es, device, batch_size=4000)

Loading data...
Data loaded!
Loading data...
Data loaded!


In [22]:
test_en

In [24]:
src_pad = SRC.vocab.stoi['<pad>']
tgt_pad = TGT.vocab.stoi['<pad>']
criterion = LabelSmoothing(
    size=len(TGT.vocab), padding_idx=tgt_pad, smoothing=0.1).to(device)
optimizer = DynamicOptimizer(en_model.core.d_model, 1, 2000,
                              torch.optim.Adam(en_model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))
compute_loss = LossCompute(en_model.generator, criterion, optimizer)

en_model.eval()
en_valid_loss = run_epoch(
    en_model, (make_batch(b, src_pad, tgt_pad, device)
            for b in test_itr_en),
    LossCompute(en_model.generator, criterion, None), 100, device)

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


Step   100 : 582.07 ms/batch | loss  3.22 | perplexity 25.03
Step   200 : 549.25 ms/batch | loss  3.46 | perplexity 31.87
Step   300 : 564.84 ms/batch | loss  3.17 | perplexity 23.81
Step   400 : 559.92 ms/batch | loss  3.31 | perplexity 27.42
Step   500 : 559.09 ms/batch | loss  2.80 | perplexity 16.52
Step   600 : 559.13 ms/batch | loss  3.23 | perplexity 25.27


In [25]:
criterion = LabelSmoothing(
    size=len(SRC.vocab), padding_idx=src_pad, smoothing=0.1).to(device)
optimizer = DynamicOptimizer(es_model.core.d_model, 1, 2000,
                              torch.optim.Adam(es_model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))
compute_loss = LossCompute(es_model.generator, criterion, optimizer)

es_model.eval()
es_valid_loss = run_epoch(
    es_model, (make_batch(b, tgt_pad, src_pad, device)
            for b in test_itr_es),
    LossCompute(es_model.generator, criterion, None), 100, device)

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Step   100 : 578.80 ms/batch | loss  3.42 | perplexity 30.54
Step   200 : 559.77 ms/batch | loss  3.56 | perplexity 35.31
Step   300 : 568.90 ms/batch | loss  3.37 | perplexity 29.18
Step   400 : 552.22 ms/batch | loss  3.13 | perplexity 22.77
Step   500 : 561.16 ms/batch | loss  3.11 | perplexity 22.40
Step   600 : 553.34 ms/batch | loss  3.63 | perplexity 37.54


In [27]:
print("Modelo EN-ES: loss {}, perplexity {}".format(en_valid_loss, math.exp(en_valid_loss)))
print("Modelo ES-EN: loss {}, perplexity {}".format(es_valid_loss, math.exp(es_valid_loss)))

Modelo EN-ES: loss 3.286382731767633, perplexity 26.74594122083582
Modelo ES-EN: loss 3.4734340892962194, perplexity 32.247292598905716


### Translate some sentences

In [28]:
sos = SRC.vocab.stoi['<s>']
eos = SRC.vocab.stoi['</s>']
translate_dataset(en_model, test_en, SRC, TGT, sos, eos, tgt_pad, device, 5)

Src:  <s>Because, what, hopefully, will, be, some, form, of, solution, will, be, in, the, nature, of, a, compromise, ,, I, also, think, it, would, be, entirely, reasonable, to, establish, a, deadline, ,, for, example, a, couple, of, years, from, now, ,, by, which, this, compromise, should, have, been, evaluated, so, that, we, can, really, see, if, it, is, operating, well, and, ,, if, it, is, not, ,, can, develop, and, improve, it, .</s>
Candidates:
Translation: <s>, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿ - score: -131.986083984375
Translation: <s>, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, ¿, Muchas, ¿, ¿, ¿, ¿, ¿, ¿ - score: -132.8208818435669
Translation: <s>, ¿, 